In [ ]:
from pyq import q,K
from kiteconnect import KiteConnect
import datetime
from kiteconnect import KiteTicker
import os
import pandas as pd
import logging
import time
    
logging.basicConfig(level=logging.DEBUG)
print("===== imported all modules ===== ")
               
instrument_token=[5633, 6401, 3861249, 4451329, 25601, 325121, 40193, 41729, 54273, 60417, 70401, 1510401, 4268801, 81153, 85761, 579329, 1195009, 94977, 98049, 103425, 108033, 2714625, 112129, 2911489, 558337, 134657, 140033, 2029825, 2763265, 160001, 160769, 175361, 177665, 5215745, 3876097, 1215745, 486657, 197633, 2800641, 3771393, 225537, 232961, 4314113, 245249, 173057, 261889, 1207553, 1895937, 3463169, 2585345, 315393, 2513665, 1850625, 340481, 341249, 119553, 345089, 348929, 359937, 356865, 7712001, 1270529, 4774913, 3677697, 2863105, 2883073, 2865921, 1346049, 7458561, 408065, 415745, 424961, 1723649, 3001089, 4632577, 7670273, 492033, 511233, 2939649, 2672641, 4879617, 1041153, 2815745, 548353, 4488705, 3675137, 1076225, 582913, 6054401, 1629185, 3520257, 593665, 4598529, 2955009, 3924993, 2977281, 4464129, 633601, 3689729, 617473, 2905857, 3660545, 681985, 2730497, 3834113, 3365633, 523009, 4708097, 3930881, 738561, 758529, 779521, 794369, 806401, 837889, 1102337, 857857, 3431425, 871681, 884737, 877057, 895745, 2953217, 3465729, 897537, 900609, 3529217, 2170625, 4278529, 4369665, 2952193, 2889473, 784129, 951809, 969473, 975873]
userdata = pd.read_csv("C://db//loginkey//userdata.csv")
    
class Mdatabase:
    # date must be chaged manually
    def __init__(self):
        self.defineqfunc()
           
    def defineqfunc(self):
        todate=datetime.date.today().strftime('%Y.%m.%d')
        q.todate = todate
        
        
        %q tcol:`instrument_token`last_price`last_quantity`average_price`volume`buy_quantity`sell_quantity`open`high`low`close`change`last_trade_time`oi`oi_day_high`oi_day_low`time`bdq1`bdp1`bdo1`bdq2`bdp2`bdo2`bdq3`bdp3`bdo3`bdq4`bdp4`bdo4`bdq5`bdp5`bdo5`sdq1`sdp1`sdo1`sdq2`sdp2`sdo2`sdq3`sdp3`sdo3`sdq4`sdp4`sdo4`sdq5`sdp5`sdo5
        %q tschema:([] instrument_token:`long$();last_price:`float$();last_quantity:`long$();average_price:`float$();volume:`long$();buy_quantity:`long$();sell_quantity:`long$();open:`float$();high:`float$();low:`float$();close:`float$();change:`float$();last_trade_time:`time$();oi:`long$();oi_day_high:`long$();oi_day_low:`long$();time:`time$();bdq1:`long$();bdp1:`float$();bdo1:`long$();bdq2:`long$();bdp2:`float$();bdo2:`long$();bdq3:`long$();bdp3:`float$();bdo3:`long$();bdq4:`long$();bdp4:`float$();bdo4:`long$();bdq5:`long$();bdp5:`float$();bdo5:`long$();sdq1:`long$();sdp1:`float$();sdo1:`long$();sdq2:`long$();sdp2:`float$();sdo2:`long$();sdq3:`long$();sdp3:`float$();sdo3:`long$();sdq4:`long$();sdp4:`float$();sdo4:`long$();sdq5:`long$();sdp5:`float$();sdo5:`long$())
        # %q loadmdata:{[x] lst:string x; lst:parse lst; lst:1_lst; lst:47 cut lst; tab:flip tcol!(flip lst); `:/db/mdata/2018.07.06/quote/ upsert tab}
        %q loadmdata:{[x] lst:string x; lst:parse lst; lst:1_lst; lst:47 cut lst; tab:flip tcol!(flip lst); d:string todate; hpath:hsym`$"/kdb/data/",d,"/quote/";hpath upsert tab}

    def upertmdata(self,data):
        try:
            q.qmdata="{}".format(data)
            %q loadmdata[qmdata]
            #print("{} Mdatabase upated ".format(datetime.datetime.now()))
        except Exception as e:
            print("Error in Mdatabase upertmdata ::  {}".format(e))

    def updatepositions(self, netp, dayp):
        try:
            print("{} Positions updated".format(datetime.datetime.now()))
        except Exception as e:
            print("Error in upsert data {}".format(e))

mdatabase= Mdatabase()

class KiteWebSocket:
    def __init__(self):
        KiteWebSocket.row=[]
        self.lasttick = time.time()
        self.wsobj=None

        self.api_key = userdata.loc[0, "api_key"]
        self.api_secret = userdata.loc[0, "api_secret"]
        self.request_token = userdata.loc[0, "request_token"]
        self.access_token = userdata.loc[0, "access_token"]
        self.public_token = userdata.loc[0, "public_token"]

        kite = KiteConnect(api_key=self.api_key)
        kite.set_access_token(self.access_token)

    def flatten(data):
        for key, value in data.items():
            if isinstance(value, dict):
                KiteWebSocket.flatten(value)
            elif isinstance(value, list):
                [KiteWebSocket.flatten(i) for i in value]
            else:
                if isinstance(value, datetime.datetime):
                    value=value.strftime('%H:%M:%S.%f')[:-3]
                elif value==None:
                    value=datetime.datetime.now().strftime('%H:%M:%S.%f')[:-3]
                KiteWebSocket.row.append(value)

    def on_ticks(ws, ticks):
        #logging.debug("\n{}\n".format(ticks))
        try:
            KiteWebSocket.row = []
            datalen=len(ticks)
            for value in ticks:
                del value['tradable']
                del value['mode']
                KiteWebSocket.flatten(value)
            data=';'.join(map(str, KiteWebSocket.row))
            logging.debug("==={}===:{}".format(datalen, datetime.datetime.now()))
            #logging.debug("==={}===".format(datalen))
            mdatabase.upertmdata(data)
        except Exception as e:
            logging.debug("===on_ticks Exception === ".format(e))

    def on_connect(ws, response):
        logging.debug("=====================Connected to websocket  Responcse is {} =====================\n\n".format(response))
        tokens = instrument_token
        ws.subscribe(tokens)
        ws.set_mode(ws.MODE_FULL, tokens)

    def on_close(ws, code, reason):
        logging.debug(" KiteWebSocket Closed reason :{} XXXXXXXXXXXX ".format(reason))
        #ws.stop() # this must be removed 

    def on_error(ws, code, reason):
        logging.debug("\n\n\n ==={}===".format(datetime.datetime.now()))
        logging.debug("KiteWebSocket on_error, reason: {}".format(reason))

    def startrun(self):
        kws = KiteTicker(self.api_key, self.access_token)
        kws.on_ticks = KiteWebSocket.on_ticks
        kws.on_connect = KiteWebSocket.on_connect
        kws.on_close = KiteWebSocket.on_close
        kws.on_error = KiteWebSocket.on_error
        logging.debug("KiteWebSocket Class initialized")

        try:
            kws.connect(disable_ssl_verification=True)
            kws.CONNECT_TIMEOUT=600
            kws.RECONNECT_MAX_DELAY=600
            kws.RECONNECT_MAX_TRIES = 20000

        except Exception as e:
            logging.debug("KiteWebSocket Exception {}".format(e))

    def __del__(self):
        try:
            self.wsobj.stop()
            pass
        except Exception:
            pass
        logging.debug("XXXXXXXXXX self.wsobj.stop() XXXXXXX")

def run():
    start_time = int(9) * 60 + int(14)  
    end_time = int(15) * 60 + int(30)
    runcount=0
    while True:
        if (datetime.datetime.now().hour * 60 + datetime.datetime.now().minute) == end_time:
            print("Currenet time is higher than end time : Break")
            break
        elif (datetime.datetime.now().hour * 60 + datetime.datetime.now().minute) > start_time:
            logging.debug("\n\n\n\n===========Connecting to KiteWebSocket, Count {}, {} ==========".format(runcount,datetime.datetime.now()))
            try:
                wsinstance = KiteWebSocket()
                wsinstance.startrun()
            except Exception as e:
                logging.debug("\n\n\n\n===========Error on Connecting to KiteWebSocket, Exception {}".format(e))
            runcount=runcount+1
            time.sleep(10)
        else:
            print('Waiting...', datetime.datetime.now())
            time.sleep(1)
                
print("===== All Clases are defined ===== ")

run()